<a href="https://colab.research.google.com/github/tirals88/pytorch-drl/blob/main/DRL_Chap3_2_CartPole_Q_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

'PyTorch를 활용한 강화학습/심층강화학습 실전입문' 책 스터디 내용을 Google Colab으로 정리하여 올립니다.

Github 예제 코드 주소 : 'https://github.com/wikibook/pytorch-drl'

PyTorch를 활용한 강화학습/심층강화학습 실전입문

https://wikibook.co.kr/pytorch-drl/

### ***가상환경 구현 및 함수 수정***

In [ ]:
# matplotlib -downgrade

!pip install matplotlib==3.4.2

In [ ]:
!apt update
!apt install -y xvfb #server install
import sys

IS_COLAB = "google.colab" in sys.modules

if IS_COLAB:
  !apt update && apt install -y libpq-dev libsdl2-dev swig xorg-dev xvfb
  %pip install -U tf-agents pyvirtualdisplay
  %pip install -U gym>=0.21.0
  %pip install -U gym[box2d,atari, accept-rom-license]

!xvfb-run -s "-screen 0 1400x900x24" jupyter notebook

### ***display_frames_as_gif 함수 정의***

In [ ]:
# 구현에 사용할 패키지 임포트
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import gym

In [ ]:
# 애니메이션을 만드는 함수
# 참고 URL : https://github.com/patrickmineault/xcorr-notebooks/blob/master/notebooks/Render%20OpenAI%20gym%20as%20GIF.ipynb
!pip install JSAnimation
from JSAnimation.IPython_display import display_animation
from IPython.display import display

#새로 정의한 display_animtaion 함수

from IPython.display import HTML

def display_animation(anim):
  return HTML(anim.to_jshtml())

In [ ]:
def display_frames_as_gif(frames):
  """
  Displays a list of frames as a gif, with controls
  """
  plt.figure(figsize=(frames[0].shape[1]/48.0, frames[0].shape[0]/48.0), dpi=72)
  patch = plt.imshow(frames[0])
  plt.axis('off')

  def animate(i):
    patch.set_data(frames[i])

  anim = animation.FuncAnimation(plt.gcf(), animate, frames=len(frames), interval=20)

  anim.save('movie_cartpole.mp4') #애니메이션을 저장하는 부분
  display(display_animation(anim)) #수정된 부분

## 3.3 다변수, 연속값 상태를 표형식으로 나타내기

### CartPole 의 상태

이전 장에서 다뤘던 미로 태스크에서 상태는 에이전트가 어느 칸에 위치했는지를 변수 하나로 나타냈으며 0~8의 단순 이산값이었다. 그러나 역진자 태스크에서는 더 복잡하게 상태가 정의된다.

CartPole 태스크는 observation 변수에 상태를 저장한다. env.step(action)은 게임 환경을 1단계 진행시키는 명령어로 action = 0 일 때 수레를 왼쪽으로 움직이는 것이고 1 일 때 오른쪽으로 움직이는 것에 해당한다.

env.step(action) 은 observation, reward, done, info 라는 4개의 변수를 반환한다. observation은 수레와 봉의 상태를 나타내며 상태는 다시 4개의 변수로 이루어져 있다. 이는 수레의 위치 (-2.4 ~ 2.4), 수레의 속도 (-inf ~ inf), 봉의 각도 (-41.8도 ~ 41.8), 봉의 가속도 (-inf ~ inf)이다.

reward는 즉각보상이며 action을 실행한 후에 수레의 위치가 $\pm2.4$ 범위 안에 있고 봉이 $20.9$도 이상 기울어 있지 않다면 보상 1을 받는다. 반대의 경우, 보상은 0이 된다.

done 은 게임 종료 여부를 나타내며, 200단계를 초과하거나, 보상이 0일 될 때 게임이 종료되어 done 의 값이 True가 된다.

info 는 디버깅 등에 필요한 정보를 담은 변수이다.